@Copyright IQIYI 2021
http://challenge.ai.iqiyi.com/

In [1]:
import pandas as pd
import numpy as np
from itertools import groupby

In [2]:
input_dir = "~/wsdm_raw_data/"
output_dir = "~/wsdm_model_data/"

In [3]:
launch = pd.read_csv(input_dir + "app_launch_logs.csv")
launch.date.min(), launch.date.max()

(100, 222)

In [4]:
launch_grp = launch.groupby("user_id").agg(
    launch_date=("date", list),
    launch_type=("launch_type", list)
).reset_index()
launch_grp

,user_id,launch_date,launch_type
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000001,"[118, 117, 114]","[0, 0, 0]"
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]"
3,10000003,"[144, 144]","[1, 0]"
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]"
...,...,...,...
599995,10599995,"[196, 174, 197, 190, 189, 155, 201, 175, 165, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
599996,10599996,"[185, 209, 207, 187, 189, 213, 184, 199, 210, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
599997,10599997,"[115, 159, 160, 166, 161, 157, 163, 158, 134, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
599998,10599998,"[137, 160, 182, 159, 107]","[0, 0, 0, 0, 0]"


In [5]:
# generating a sample for each user, a sample should has an anchor date for label
def choose_end_date(launch_date):
    n1, n2 = min(launch_date), max(launch_date)
    if n1 < n2 - 7:
        end_date = np.random.randint(n1, n2 - 7)
    else:
        end_date = np.random.randint(100, 222 - 7)
    return end_date
launch_grp["end_date"] = launch_grp.launch_date.apply(choose_end_date)
launch_grp

,user_id,launch_date,launch_type,end_date
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",205
1,10000001,"[118, 117, 114]","[0, 0, 0]",104
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",132
3,10000003,"[144, 144]","[1, 0]",188
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",152
...,...,...,...,...
599995,10599995,"[196, 174, 197, 190, 189, 155, 201, 175, 165, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",178
599996,10599996,"[185, 209, 207, 187, 189, 213, 184, 199, 210, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",198
599997,10599997,"[115, 159, 160, 166, 161, 157, 163, 158, 134, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",161
599998,10599998,"[137, 160, 182, 159, 107]","[0, 0, 0, 0, 0]",122


In [6]:
def get_label(row):
    launch_list = row.launch_date
    end = row.end_date
    label = sum([1 for x in set(launch_list) if end < x < end+8])
    return label
launch_grp["label"] = launch_grp.apply(get_label, axis=1)
launch_grp

,user_id,launch_date,launch_type,end_date,label
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",205,1
1,10000001,"[118, 117, 114]","[0, 0, 0]",104,0
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",132,0
3,10000003,"[144, 144]","[1, 0]",188,0
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",152,4
...,...,...,...,...,...
599995,10599995,"[196, 174, 197, 190, 189, 155, 201, 175, 165, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",178,0
599996,10599996,"[185, 209, 207, 187, 189, 213, 184, 199, 210, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",198,1
599997,10599997,"[115, 159, 160, 166, 161, 157, 163, 158, 134, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",161,4
599998,10599998,"[137, 160, 182, 159, 107]","[0, 0, 0, 0, 0]",122,0


In [7]:
launch_grp.label.value_counts()

0    303093
1    126052
2     57792
3     33721
4     23336
7     20389
5     18846
6     16771
Name: label, dtype: int64

In [9]:
train = launch_grp[["user_id", "end_date", "label"]]
train

,user_id,end_date,label
0,10000000,205,1
1,10000001,104,0
2,10000002,132,0
3,10000003,188,0
4,10000004,152,4
...,...,...,...
599995,10599995,178,0
599996,10599996,198,1
599997,10599997,161,4
599998,10599998,122,0


In [10]:
# reading test data
test = pd.read_csv(input_dir + "test-a.csv")
test["label"] = -1
test

,user_id,end_date,label
0,10007813,205,-1
1,10052988,210,-1
2,10279068,200,-1
3,10546696,216,-1
4,10406659,183,-1
...,...,...,...
14996,10355586,205,-1
14997,10589773,210,-1
14998,10181954,218,-1
14999,10544736,164,-1


In [11]:
# concat train and test data
data = pd.concat([train, test], ignore_index=True)
data

,user_id,end_date,label
0,10000000,205,1
1,10000001,104,0
2,10000002,132,0
3,10000003,188,0
4,10000004,152,4
...,...,...,...
614996,10355586,205,-1
614997,10589773,210,-1
614998,10181954,218,-1
614999,10544736,164,-1


# launch data process

In [12]:
# append test data to launch_grp
launch_grp = launch_grp.append(
    test.merge(launch_grp[["user_id", "launch_type", "launch_date"]], how="left", on="user_id")
)
launch_grp

,user_id,launch_date,launch_type,end_date,label
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",205,1
1,10000001,"[118, 117, 114]","[0, 0, 0]",104,0
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",132,0
3,10000003,"[144, 144]","[1, 0]",188,0
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",152,4
...,...,...,...,...,...
14996,10355586,"[118, 115, 124]","[0, 1, 0]",205,-1
14997,10589773,"[183, 144, 173, 167, 203, 204, 178, 155, 177, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",210,-1
14998,10181954,"[152, 155, 153, 144, 150, 143, 158, 151, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",218,-1
14999,10544736,"[138, 117, 105, 148, 113, 116, 115, 158, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164,-1


In [13]:
# get latest 32 days([end_date-31, end_date]) launch type sequence
# 0 for not launch, 1 for launch_type=0, and 2 for launch_type=1
def gen_launch_seq(row):
    seq_sort = sorted(zip(row.launch_type, row.launch_date), key=lambda x: x[1])
    seq_map = {k: max(g)[0] + 1 for k, g in groupby(seq_sort, lambda x: x[1])}
    end = row.end_date
    seq = [seq_map.get(x, 0) for x in range(end-31, end+1)]
    return seq
launch_grp["launch_seq"] = launch_grp.apply(gen_launch_seq, axis=1)
launch_grp

,user_id,launch_date,launch_type,end_date,label,launch_seq
0,10000000,"[195, 192, 196, 193, 191, 141, 187, 202, 132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ..."
1,10000001,"[118, 117, 114]","[0, 0, 0]",104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10000002,"[129, 144, 128, 140, 163, 160]","[0, 0, 0, 0, 0, 0]",132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,10000003,"[144, 144]","[1, 0]",188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10000004,"[157, 154, 155, 156]","[0, 0, 0, 0]",152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
14996,10355586,"[118, 115, 124]","[0, 1, 0]",205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14997,10589773,"[183, 144, 173, 167, 203, 204, 178, 155, 177, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
14998,10181954,"[152, 155, 153, 144, 150, 143, 158, 151, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14999,10544736,"[138, 117, 105, 148, 113, 116, 115, 158, 154, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
data = data.merge(
    launch_grp[["user_id", "end_date", "label", "launch_seq"]],
    on=["user_id", "end_date", "label"],
    how="left"
)
data

,user_id,end_date,label,launch_seq
0,10000000,205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ..."
1,10000001,104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10000002,132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,10000003,188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10000004,152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
614996,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
614997,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ..."
614998,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
614999,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# playback data and video data process

In [16]:
# choose playback data in [end_date-31, end_date]
playback = pd.read_csv(input_dir + "user_playback_data.csv", dtype={"item_id": str})
playback = playback.merge(data, how="inner", on="user_id")
playback = playback.loc[(playback.date >= playback.end_date-31) & (playback.date <= playback.end_date)]
playback

,user_id,item_id,playtime,date,end_date,label,launch_seq
0,10057286,20628283,2208.612,145,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,10057286,20628283,332.351,150,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
5,10057286,24986210,74.370,161,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
6,10057286,23148200,110.134,145,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
8,10057286,23821440,1458.661,161,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
73564812,10184651,24353733,11.343,144,148,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
73564814,10054781,20448293,37.835,144,164,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, ..."
73564820,10037173,23616810,1.936,144,159,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
73564821,10450556,23026777,3209.858,144,160,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [17]:
# add video info to playback data
video_data = pd.read_csv(input_dir + "video_related_data.csv", dtype=str)
playback = playback.merge(video_data[video_data.item_id.notna()], how="left", on="item_id")
playback

,user_id,item_id,playtime,date,end_date,label,launch_seq,duration,father_id,tag_list,cast
0,10057286,20628283,2208.612,145,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",11,30038149,50444644;50260120;50057164;50323629;50107634;5...,40015213;40019805;40021027;40005903;40004251;4...
1,10057286,20628283,332.351,150,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",11,30038149,50444644;50260120;50057164;50323629;50107634;5...,40015213;40019805;40021027;40005903;40004251;4...
2,10057286,24986210,74.370,161,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",NaN,NaN,NaN,NaN
3,10057286,23148200,110.134,145,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",NaN,NaN,NaN,NaN
4,10057286,23821440,1458.661,161,165,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
22356866,10184651,24353733,11.343,144,148,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6,NaN,50152648;50367774;50323515;50057231;50330872,NaN
22356867,10054781,20448293,37.835,144,164,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, ...",NaN,NaN,NaN,NaN
22356868,10037173,23616810,1.936,144,159,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN
22356869,10450556,23026777,3209.858,144,160,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",NaN,NaN,NaN,NaN


In [18]:
# using target encoding
# Tutorial: https://www.kaggle.com/ryanholbrook/target-encoding
def target_encoding(name, df, m=1):
    df[name] = df[name].str.split(";")
    df = df.explode(name)
    overall = df["label"].mean()
    df = df.groupby(name).agg(
        freq=("label", "count"), 
        in_category=("label", np.mean)
    ).reset_index()
    df["weight"] = df["freq"] / (df["freq"] + m)
    df["score"] = df["weight"] * df["in_category"] + (1 - df["weight"]) * overall
    return df

In [19]:
# father_id target encoding
df = playback.loc[(playback.label >= 0) & (playback.father_id.notna()), ["father_id", "label"]]
father_id_score = target_encoding("father_id", df)
father_id_score

,father_id,freq,in_category,weight,score
0,30000000,5,4.800000,0.833333,4.671506
1,30000001,1,6.000000,0.500000,5.014517
2,30000002,7,2.571429,0.875000,2.753629
3,30000004,1,7.000000,0.500000,5.514517
4,30000007,1,5.000000,0.500000,4.514517
...,...,...,...,...,...
60626,30094642,7,1.142857,0.875000,1.503629
60627,30094643,39,3.230769,0.975000,3.250726
60628,30094644,1,0.000000,0.500000,2.014517
60629,30094645,13,4.230769,0.928571,4.216360


In [20]:
# tag_id target encoding
df = playback.loc[(playback.label >= 0) & (playback.tag_list.notna()), ["tag_list", "label"]]
tag_id_score = target_encoding("tag_list", df)
tag_id_score.rename({"tag_list": "tag_id"}, axis=1, inplace=True)
tag_id_score

,tag_id,freq,in_category,weight,score
0,50000000,2,6.000000,0.666667,5.319083
1,50000001,9,2.111111,0.900000,2.295725
2,50000003,5,4.000000,0.833333,3.992875
3,50000004,6,3.000000,0.857143,3.136750
4,50000005,1,7.000000,0.500000,5.478624
...,...,...,...,...,...
267234,50448550,2,2.000000,0.666667,2.652416
267235,50448551,12,3.416667,0.923077,3.458250
267236,50448552,3,4.666667,0.750000,4.489312
267237,50448553,1,0.000000,0.500000,1.978624


In [21]:
# cast_id target encoding
df = playback.loc[(playback.label >= 0) & (playback.cast.notna()), ["cast", "label"]]
cast_id_score = target_encoding("cast", df)
cast_id_score.rename({"cast": "cast_id"}, axis=1, inplace=True)
cast_id_score

,cast_id,freq,in_category,weight,score
0,40000000,2,4.000000,0.666667,3.935279
1,40000001,1508,2.942971,0.999337,2.943543
2,40000004,93,3.698925,0.989362,3.700062
3,40000005,19,2.842105,0.950000,2.890292
4,40000009,10,4.100000,0.909091,4.073258
...,...,...,...,...,...
26821,40041230,175,3.920000,0.994318,3.919351
26822,40041231,145,4.048276,0.993151,4.046615
26823,40041235,6,4.166667,0.857143,4.115120
26824,40041236,101,1.742574,0.990196,1.762802


In [22]:
# group playback data for feature engineering
playback_grp = playback.groupby(["user_id", "end_date", "label"]).agg(
    playtime_list=("playtime", list),
    date_list=("date", list),
    duration_list=("duration", lambda x: ";".join(map(str, x))),
    father_id_list=("father_id", lambda x: ";".join(map(str, x))),
    tag_list=("tag_list", lambda x: ";".join(map(str, x))),
    cast_list=("cast", lambda x: ";".join(map(str, x)))
).reset_index()
playback_grp

,user_id,end_date,label,playtime_list,date_list,duration_list,father_id_list,tag_list,cast_list
0,10000000,205,1,"[2567.688, 19.482]","[195, 195]",nan;11,nan;30062196,nan;50409449;50325616;50260120;50182876;501680...,nan;40012204;40023040;40024229;40003755;400015...
1,10000002,132,0,[19.925],[128],12,nan,50337119;50395963;50329038;50413883;50436897;5...,40023838;40026046;40011147;40022185;40029352;4...
2,10000005,144,2,"[84.996, 36.409, 2.121, 2.555, 1475.583]","[142, 142, 142, 142, 124]",7;6;7;7;nan,nan;nan;nan;30084288;nan,50181045;50186586;50017761;50260245;50386222;5...,nan;nan;40010951;40018503;40038873;40022107;40...
3,10000007,155,0,"[4.889, 2.222, 181.756, 648.366, 19.32, 4902.9...","[151, 151, 151, 151, 151, 151, 151, 151]",nan;nan;nan;nan;6;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;50334950;50220609;50200580;501...,nan;nan;nan;nan;nan;nan;nan;nan
4,10000009,147,3,"[3.073, 1.2, 3.128, 2.607, 1050.362, 1.401, 10...","[147, 147, 147, 147, 147, 147, 147, 147, 147, ...",nan;11;11;nan;nan;nan;11;nan;nan;11;nan;8;nan;...,nan;30060857;30079688;nan;nan;nan;30079688;nan...,nan;50409449;50260120;50325616;50420012;502949...,nan;40023053;40033484;40015875;40008370;400168...
...,...,...,...,...,...,...,...,...,...
424527,10599994,182,1,"[6.331, 49.379, 502.054]","[151, 165, 151]",nan;nan;9,nan;nan;30057840,nan;nan;50401854;50294976;50096449;50207929;50...,nan;nan;nan
424528,10599995,178,0,"[536.882, 32.337, 110.372, 1.164, 3.896, 79.12...","[174, 155, 155, 155, 174, 155, 155, 165, 155, ...",nan;nan;9;nan;nan;11;nan;nan;9;10;8;9;6;6;nan;...,nan;nan;nan;nan;nan;30064457;nan;nan;nan;30001...,nan;nan;50396386;50351235;50353717;50432753;50...,nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;na...
424529,10599996,198,1,"[97.934, 68.63, 2006.14, 3085.066, 1637.014, 2...","[184, 184, 189, 185, 187, 185]",12;11;nan;11;11;11,30011287;30018127;nan;30018127;30018127;30018127,50228057;50382939;50413323;50310754;50161432;5...,40017579;40021254;40029288;40039051;40018789;4...
424530,10599997,161,4,"[4.302, 1106.0, 1.48, 5.516, 27.37, 1841.562, ...","[157, 158, 159, 157, 134, 160, 157, 157, 158, ...",nan;nan;nan;nan;6;11;nan;11;nan;11;nan;nan;11;...,nan;nan;nan;nan;30019318;30032552;nan;30032552...,nan;nan;nan;nan;50195851;50003290;50147017;500...,nan;nan;nan;nan;nan;40017665;40030684;nan;4001...


In [23]:
# generate latest 32 days([end_date-31, end_date]) playtime sequence
# playtime_norm = 1/(1 + exp(3 - playtime/450)). when playtime=3600s it's preference score is almost equal to 1
def get_playtime_seq(row):
    seq_sort = sorted(zip(row.playtime_list, row.date_list), key=lambda x: x[1])
    seq_map = {k: sum(x[0] for x in g) for k, g in groupby(seq_sort, key=lambda x: x[1])}
    seq_norm = {k: 1/(1+np.exp(3-v/450)) for k, v in seq_map.items()}
    seq = [round(seq_norm.get(i, 0), 4) for i in range(row.end_date-31, row.end_date+1)]
    return seq
playback_grp["playtime_seq"] = playback_grp.apply(get_playtime_seq, axis=1)
playback_grp

,user_id,end_date,label,playtime_list,date_list,duration_list,father_id_list,tag_list,cast_list,playtime_seq
0,10000000,205,1,"[2567.688, 19.482]","[195, 195]",nan;11,nan;30062196,nan;50409449;50325616;50260120;50182876;501680...,nan;40012204;40023040;40024229;40003755;400015...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10000002,132,0,[19.925],[128],12,nan,50337119;50395963;50329038;50413883;50436897;5...,40023838;40026046;40011147;40022185;40029352;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10000005,144,2,"[84.996, 36.409, 2.121, 2.555, 1475.583]","[142, 142, 142, 142, 124]",7;6;7;7;nan,nan;nan;nan;30084288;nan,50181045;50186586;50017761;50260245;50386222;5...,nan;nan;40010951;40018503;40038873;40022107;40...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5693, 0, 0..."
3,10000007,155,0,"[4.889, 2.222, 181.756, 648.366, 19.32, 4902.9...","[151, 151, 151, 151, 151, 151, 151, 151]",nan;nan;nan;nan;6;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;50334950;50220609;50200580;501...,nan;nan;nan;nan;nan;nan;nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,10000009,147,3,"[3.073, 1.2, 3.128, 2.607, 1050.362, 1.401, 10...","[147, 147, 147, 147, 147, 147, 147, 147, 147, ...",nan;11;11;nan;nan;nan;11;nan;nan;11;nan;8;nan;...,nan;30060857;30079688;nan;nan;nan;30079688;nan...,nan;50409449;50260120;50325616;50420012;502949...,nan;40023053;40033484;40015875;40008370;400168...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...
424527,10599994,182,1,"[6.331, 49.379, 502.054]","[151, 165, 151]",nan;nan;9,nan;nan;30057840,nan;nan;50401854;50294976;50096449;50207929;50...,nan;nan;nan,"[0.1335, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
424528,10599995,178,0,"[536.882, 32.337, 110.372, 1.164, 3.896, 79.12...","[174, 155, 155, 155, 174, 155, 155, 165, 155, ...",nan;nan;9;nan;nan;11;nan;nan;9;10;8;9;6;6;nan;...,nan;nan;nan;nan;nan;30064457;nan;nan;nan;30001...,nan;nan;50396386;50351235;50353717;50432753;50...,nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;na...,"[0, 0, 0, 0, 0, 0, 0, 0, 0.9999, 0, 0, 0, 0, 0..."
424529,10599996,198,1,"[97.934, 68.63, 2006.14, 3085.066, 1637.014, 2...","[184, 184, 189, 185, 187, 185]",12;11;nan;11;11;11,30011287;30018127;nan;30018127;30018127;30018127,50228057;50382939;50413323;50310754;50161432;5...,40017579;40021254;40029288;40039051;40018789;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
424530,10599997,161,4,"[4.302, 1106.0, 1.48, 5.516, 27.37, 1841.562, ...","[157, 158, 159, 157, 134, 160, 157, 157, 158, ...",nan;nan;nan;nan;6;11;nan;11;nan;11;nan;nan;11;...,nan;nan;nan;nan;30019318;30032552;nan;30032552...,nan;nan;nan;nan;50195851;50003290;50147017;500...,nan;nan;nan;nan;nan;40017665;40030684;nan;4001...,"[0, 0, 0, 0, 0.0503, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [24]:
drn_desc = video_data.loc[video_data.duration.notna(), "duration"].astype(int)
drn_desc.min(), drn_desc.max()

(1, 16)

In [25]:
# duration preference is a 16-dimentional prefer vector
# for a user, count the frequency of each duration
# prefer_score = freq / max(freq)
# if the user's duration_list is all null, then return null
# null duration_prefer will later be filled with zero vector
def get_duration_prefer(duration_list):
    drn_list = sorted(duration_list.split(";"))
    drn_map = {k: sum(1 for _ in g) for k, g in groupby(drn_list) if k != "nan"}
    if drn_map:
        max_ = max(drn_map.values())
        res = [round(drn_map.get(str(i), 0)/max_, 4) for i in range(1, 17)]
        return res
    else:
        return np.nan
playback_grp["duration_prefer"] = playback_grp.duration_list.apply(get_duration_prefer)

,user_id,end_date,label,playtime_list,date_list,duration_list,father_id_list,tag_list,cast_list,playtime_seq,duration_prefer
0,10000000,205,1,"[2567.688, 19.482]","[195, 195]",nan;11,nan;30062196,nan;50409449;50325616;50260120;50182876;501680...,nan;40012204;40023040;40024229;40003755;400015...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,10000002,132,0,[19.925],[128],12,nan,50337119;50395963;50329038;50413883;50436897;5...,40023838;40026046;40011147;40022185;40029352;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,10000005,144,2,"[84.996, 36.409, 2.121, 2.555, 1475.583]","[142, 142, 142, 142, 124]",7;6;7;7;nan,nan;nan;nan;30084288;nan,50181045;50186586;50017761;50260245;50386222;5...,nan;nan;40010951;40018503;40038873;40022107;40...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5693, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.3333, 1.0, 0.0, 0...."
3,10000007,155,0,"[4.889, 2.222, 181.756, 648.366, 19.32, 4902.9...","[151, 151, 151, 151, 151, 151, 151, 151]",nan;nan;nan;nan;6;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;50334950;50220609;50200580;501...,nan;nan;nan;nan;nan;nan;nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,10000009,147,3,"[3.073, 1.2, 3.128, 2.607, 1050.362, 1.401, 10...","[147, 147, 147, 147, 147, 147, 147, 147, 147, ...",nan;11;11;nan;nan;nan;11;nan;nan;11;nan;8;nan;...,nan;30060857;30079688;nan;nan;nan;30079688;nan...,nan;50409449;50260120;50325616;50420012;502949...,nan;40023053;40033484;40015875;40008370;400168...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1429, 0...."
...,...,...,...,...,...,...,...,...,...,...,...
424527,10599994,182,1,"[6.331, 49.379, 502.054]","[151, 165, 151]",nan;nan;9,nan;nan;30057840,nan;nan;50401854;50294976;50096449;50207929;50...,nan;nan;nan,"[0.1335, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
424528,10599995,178,0,"[536.882, 32.337, 110.372, 1.164, 3.896, 79.12...","[174, 155, 155, 155, 174, 155, 155, 165, 155, ...",nan;nan;9;nan;nan;11;nan;nan;9;10;8;9;6;6;nan;...,nan;nan;nan;nan;nan;30064457;nan;nan;nan;30001...,nan;nan;50396386;50351235;50353717;50432753;50...,nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;na...,"[0, 0, 0, 0, 0, 0, 0, 0, 0.9999, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.4286, 0.8571,..."
424529,10599996,198,1,"[97.934, 68.63, 2006.14, 3085.066, 1637.014, 2...","[184, 184, 189, 185, 187, 185]",12;11;nan;11;11;11,30011287;30018127;nan;30018127;30018127;30018127,50228057;50382939;50413323;50310754;50161432;5...,40017579;40021254;40029288;40039051;40018789;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
424530,10599997,161,4,"[4.302, 1106.0, 1.48, 5.516, 27.37, 1841.562, ...","[157, 158, 159, 157, 134, 160, 157, 157, 158, ...",nan;nan;nan;nan;6;11;nan;11;nan;11;nan;nan;11;...,nan;nan;nan;nan;30019318;30032552;nan;30032552...,nan;nan;nan;nan;50195851;50003290;50147017;500...,nan;nan;nan;nan;nan;40017665;40030684;nan;4001...,"[0, 0, 0, 0, 0.0503, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, ..."


In [26]:
# add all target encoding scores into a dict
id_score = dict()
id_score.update({x[1]: x[5] for x in father_id_score.itertuples()})
id_score.update({x[1]: x[5] for x in tag_id_score.itertuples()})
id_score.update({x[1]: x[5] for x in cast_id_score.itertuples()})

# check if features ids are duplicated
father_id_score.shape[0]+tag_id_score.shape[0]+cast_id_score.shape[0] == len(id_score)

True

In [27]:
# for these 3 features: father_id_score, cast_score, tag_score,
# choose top 3 preferences
# e.g top_3_id = [(id, freq), (id, freq), (id, freq)]
# score = weight_avg(top_3_id), which values are id_score, weights are frequency
# if the id_list is all null, then return null
def get_id_score(id_list):
    x = sorted(id_list.split(";"))
    x_count = {k: sum(1 for _ in g) for k, g in groupby(x) if k != "nan"}
    if x_count:
        x_sort = sorted(x_count.items(), key=lambda k: -k[1])
        top_x = x_sort[:3]
        res = [(n, id_score.get(k, 0)) for k, n in top_x]
        res = sum(n*v for n, v in res) / sum(n for n, v in res)
        return res
    else:
        return np.nan

In [28]:
playback_grp["father_id_score"] = playback_grp.father_id_list.apply(get_id_score)

In [29]:
playback_grp["cast_id_score"] = playback_grp.cast_list.apply(get_id_score)

In [30]:
playback_grp["tag_score"] = playback_grp.tag_list.apply(get_id_score)
playback_grp

,user_id,end_date,label,playtime_list,date_list,duration_list,father_id_list,tag_list,cast_list,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score
0,10000000,205,1,"[2567.688, 19.482]","[195, 195]",nan;11,nan;30062196,nan;50409449;50325616;50260120;50182876;501680...,nan;40012204;40023040;40024229;40003755;400015...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.197273,4.058883,3.844347
1,10000002,132,0,[19.925],[128],12,nan,50337119;50395963;50329038;50413883;50436897;5...,40023838;40026046;40011147;40022185;40029352;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,2.966641,3.162291
2,10000005,144,2,"[84.996, 36.409, 2.121, 2.555, 1475.583]","[142, 142, 142, 142, 124]",7;6;7;7;nan,nan;nan;nan;30084288;nan,50181045;50186586;50017761;50260245;50386222;5...,nan;nan;40010951;40018503;40038873;40022107;40...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5693, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.3333, 1.0, 0.0, 0....",4.137683,4.083410,3.761404
3,10000007,155,0,"[4.889, 2.222, 181.756, 648.366, 19.32, 4902.9...","[151, 151, 151, 151, 151, 151, 151, 151]",nan;nan;nan;nan;6;nan;nan;nan,nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan;nan;50334950;50220609;50200580;501...,nan;nan;nan;nan;nan;nan;nan;nan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",NaN,NaN,3.284531
4,10000009,147,3,"[3.073, 1.2, 3.128, 2.607, 1050.362, 1.401, 10...","[147, 147, 147, 147, 147, 147, 147, 147, 147, ...",nan;11;11;nan;nan;nan;11;nan;nan;11;nan;8;nan;...,nan;30060857;30079688;nan;nan;nan;30079688;nan...,nan;50409449;50260120;50325616;50420012;502949...,nan;40023053;40033484;40015875;40008370;400168...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1429, 0....",4.183522,4.083623,3.906358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424527,10599994,182,1,"[6.331, 49.379, 502.054]","[151, 165, 151]",nan;nan;9,nan;nan;30057840,nan;nan;50401854;50294976;50096449;50207929;50...,nan;nan;nan,"[0.1335, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",3.500348,NaN,3.983099
424528,10599995,178,0,"[536.882, 32.337, 110.372, 1.164, 3.896, 79.12...","[174, 155, 155, 155, 174, 155, 155, 165, 155, ...",nan;nan;9;nan;nan;11;nan;nan;9;10;8;9;6;6;nan;...,nan;nan;nan;nan;nan;30064457;nan;nan;nan;30001...,nan;nan;50396386;50351235;50353717;50432753;50...,nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;nan;na...,"[0, 0, 0, 0, 0, 0, 0, 0, 0.9999, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.4286, 0.8571,...",4.354260,3.676867,4.314434
424529,10599996,198,1,"[97.934, 68.63, 2006.14, 3085.066, 1637.014, 2...","[184, 184, 189, 185, 187, 185]",12;11;nan;11;11;11,30011287;30018127;nan;30018127;30018127;30018127,50228057;50382939;50413323;50310754;50161432;5...,40017579;40021254;40029288;40039051;40018789;4...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.423303,3.742851,3.524130
424530,10599997,161,4,"[4.302, 1106.0, 1.48, 5.516, 27.37, 1841.562, ...","[157, 158, 159, 157, 134, 160, 157, 157, 158, ...",nan;nan;nan;nan;6;11;nan;11;nan;11;nan;nan;11;...,nan;nan;nan;nan;30019318;30032552;nan;30032552...,nan;nan;nan;nan;50195851;50003290;50147017;500...,nan;nan;nan;nan;nan;40017665;40030684;nan;4001...,"[0, 0, 0, 0, 0.0503, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, ...",2.991931,3.977755,3.836918


In [31]:
data = data.merge(
    playback_grp[["user_id", "end_date", "label", "playtime_seq", "duration_prefer", "father_id_score", "cast_id_score", "tag_score"]],
    on=["user_id", "end_date", "label"],
    how="left"
)
data

,user_id,end_date,label,launch_seq,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score
0,10000000,205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.197273,4.058883,3.844347
1,10000001,104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN
2,10000002,132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,2.966641,3.162291
3,10000003,188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN
4,10000004,152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
614996,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN
614997,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.8517, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",3.033321,2.742987,3.614247
614998,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN
614999,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.0621, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",3.976856,3.995168,3.881127


# user portrait data process

In [32]:
portrait = pd.read_csv(input_dir + "user_portrait_data.csv", dtype={"territory_code": str})
portrait = pd.merge(data[["user_id", "label"]], portrait, how="left", on="user_id")
portrait

,user_id,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code
0,10000000,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313
1,10000001,0,3.0,1021,15999,2.0,3.0,2.0,0.0,863100
2,10000002,0,1.0,3117,255978,2.0,4.0,2.0,1.0,861200
3,10000003,0,2.0,3799,54204,1.0,4.0,0.0,1.0,864304
4,10000004,4,2.0,11300,463992,2.0,3.0,0.0,0.0,862301
...,...,...,...,...,...,...,...,...,...,...
614997,10355586,-1,2.0,5615,110717,1.0,2.0,2.0,0.0,866201
614998,10589773,-1,2.0,3637,53990,1.0,5.0,0.0,1.0,863503
614999,10181954,-1,2.0,7508,111125,2.0,3.0,0.0,0.0,NaN
615000,10544736,-1,3.0,1019,15999,2.0,4.0,0.0,1.0,863202


In [33]:
# for territory_code, using target encoding again
df = portrait.loc[(portrait.label >= 0) & (portrait.territory_code.notna()), ["territory_code", "label"]]
territory_score = target_encoding("territory_code", df)
territory_score

,territory_code,freq,in_category,weight,score
0,860000,1,0.000000,0.500000,0.676824
1,861100,12837,1.615019,0.999922,1.614999
2,861200,6621,1.502945,0.999849,1.502923
3,861301,4706,1.444751,0.999788,1.444732
4,861302,2879,1.372699,0.999653,1.372692
...,...,...,...,...,...
368,867107,14,1.571429,0.933333,1.556910
369,867108,38,1.657895,0.974359,1.650094
370,867109,11,0.636364,0.916667,0.696137
371,868100,126,1.158730,0.992126,1.160265


In [34]:
# add territory_code score into id_score
n1 = len(id_score)
id_score.update({x[1]: x[5] for x in territory_score.itertuples()})
n1 + territory_score.shape[0] == len(id_score)

True

In [35]:
# get territory score, retain null value
portrait["territory_score"] = portrait.territory_code.apply(lambda x: id_score.get(x, 0) if isinstance(x, str) else np.nan)
portrait

,user_id,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,territory_score
0,10000000,1,2.0,7465,112369,1.0,4.0,0.0,1.0,864313,1.126737
1,10000001,0,3.0,1021,15999,2.0,3.0,2.0,0.0,863100,1.596971
2,10000002,0,1.0,3117,255978,2.0,4.0,2.0,1.0,861200,1.502923
3,10000003,0,2.0,3799,54204,1.0,4.0,0.0,1.0,864304,1.175982
4,10000004,4,2.0,11300,463992,2.0,3.0,0.0,0.0,862301,1.316790
...,...,...,...,...,...,...,...,...,...,...,...
614997,10355586,-1,2.0,5615,110717,1.0,2.0,2.0,0.0,866201,1.311717
614998,10589773,-1,2.0,3637,53990,1.0,5.0,0.0,1.0,863503,1.627594
614999,10181954,-1,2.0,7508,111125,2.0,3.0,0.0,0.0,NaN,NaN
615000,10544736,-1,3.0,1019,15999,2.0,4.0,0.0,1.0,863202,1.580230


In [36]:
# for multi values of device_ram and device_rom, choose the first one
portrait["device_ram"] = portrait.device_ram.apply(lambda x: float(x.split(";")[0]) if isinstance(x, str) else np.nan)
portrait["device_rom"] = portrait.device_rom.apply(lambda x: float(x.split(";")[0]) if isinstance(x, str) else np.nan)
portrait

,user_id,label,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_code,territory_score
0,10000000,1,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,864313,1.126737
1,10000001,0,3.0,1021.0,15999.0,2.0,3.0,2.0,0.0,863100,1.596971
2,10000002,0,1.0,3117.0,255978.0,2.0,4.0,2.0,1.0,861200,1.502923
3,10000003,0,2.0,3799.0,54204.0,1.0,4.0,0.0,1.0,864304,1.175982
4,10000004,4,2.0,11300.0,463992.0,2.0,3.0,0.0,0.0,862301,1.316790
...,...,...,...,...,...,...,...,...,...,...,...
614997,10355586,-1,2.0,5615.0,110717.0,1.0,2.0,2.0,0.0,866201,1.311717
614998,10589773,-1,2.0,3637.0,53990.0,1.0,5.0,0.0,1.0,863503,1.627594
614999,10181954,-1,2.0,7508.0,111125.0,2.0,3.0,0.0,0.0,NaN,NaN
615000,10544736,-1,3.0,1019.0,15999.0,2.0,4.0,0.0,1.0,863202,1.580230


In [37]:
# add portrait features into data
data = data.merge(portrait.drop("territory_code", axis=1), how="left", on=["user_id", "label"])
data

,user_id,end_date,label,launch_seq,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score
0,10000000,205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.197273,4.058883,3.844347,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,1.126737
1,10000001,104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,3.0,1021.0,15999.0,2.0,3.0,2.0,0.0,1.596971
2,10000002,132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,2.966641,3.162291,1.0,3117.0,255978.0,2.0,4.0,2.0,1.0,1.502923
3,10000003,188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,3799.0,54204.0,1.0,4.0,0.0,1.0,1.175982
4,10000004,152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,11300.0,463992.0,2.0,3.0,0.0,0.0,1.316790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614997,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,5615.0,110717.0,1.0,2.0,2.0,0.0,1.311717
614998,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.8517, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",3.033321,2.742987,3.614247,2.0,3637.0,53990.0,1.0,5.0,0.0,1.0,1.627594
614999,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,7508.0,111125.0,2.0,3.0,0.0,0.0,NaN
615000,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.0621, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",3.976856,3.995168,3.881127,3.0,1019.0,15999.0,2.0,4.0,0.0,1.0,1.580230


# interaction data process

In [38]:
# only use interact_type preference
# use all interaction data to calculate interact_type preference
interact = pd.read_csv(input_dir + "user_interaction_data.csv")
interact.interact_type.min(), interact.interact_type.max()

(1, 11)

In [39]:
interact_grp = interact.groupby("user_id").agg(
    interact_type=("interact_type", list)
).reset_index()
interact_grp

,user_id,interact_type
0,10000069,"[9, 9]"
1,10000078,[10]
2,10000107,"[1, 1, 8, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,10000126,[1]
4,10000164,"[10, 10, 10]"
...,...,...
19941,10599761,"[10, 10, 9, 10, 7, 10, 7]"
19942,10599879,[7]
19943,10599887,[5]
19944,10599985,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ..."


In [40]:
# similar to duration preference, the interact_type preference could be a 11-dimentional vector
def get_interact_prefer(interact_type):
    x = sorted(interact_type)
    x_count = {k: sum(1 for _ in g) for k, g in groupby(x)}
    x_max = max(x_count.values())
    res = [round(x_count.get(i, 0)/x_max, 4) for i in range(1, 12)]
    return res
interact_grp["interact_prefer"] = interact_grp.interact_type.apply(get_interact_prefer)
interact_grp

,user_id,interact_type,interact_prefer
0,10000069,"[9, 9]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
1,10000078,[10],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,10000107,"[1, 1, 8, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 0.0, 0.0, 0.0, 0.0556, 0.0, 0.0, 0.1667,..."
3,10000126,[1],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,10000164,"[10, 10, 10]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
19941,10599761,"[10, 10, 9, 10, 7, 10, 7]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.25,..."
19942,10599879,[7],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
19943,10599887,[5],"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
19944,10599985,"[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [41]:
data = data.merge(interact_grp[["user_id", "interact_prefer"]], on="user_id", how="left")
data

,user_id,end_date,label,launch_seq,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.197273,4.058883,3.844347,2.0,7465.0,112369.0,1.0,4.0,0.0,1.0,1.126737,NaN
1,10000001,104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,3.0,1021.0,15999.0,2.0,3.0,2.0,0.0,1.596971,NaN
2,10000002,132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,2.966641,3.162291,1.0,3117.0,255978.0,2.0,4.0,2.0,1.0,1.502923,NaN
3,10000003,188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,3799.0,54204.0,1.0,4.0,0.0,1.0,1.175982,NaN
4,10000004,152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,11300.0,463992.0,2.0,3.0,0.0,0.0,1.316790,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614997,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,5615.0,110717.0,1.0,2.0,2.0,0.0,1.311717,NaN
614998,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.8517, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",3.033321,2.742987,3.614247,2.0,3637.0,53990.0,1.0,5.0,0.0,1.0,1.627594,NaN
614999,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.0,7508.0,111125.0,2.0,3.0,0.0,0.0,NaN,NaN
615000,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.0621, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",3.976856,3.995168,3.881127,3.0,1019.0,15999.0,2.0,4.0,0.0,1.0,1.580230,NaN


# feature normalization and save data

In [42]:
# the following features should be normalized
# method: x = (x - mean(x)) / std(x)
norm_cols = ["father_id_score", "cast_id_score", "tag_score", 
            "device_type", "device_ram", "device_rom", "sex",
            "age", "education", "occupation_status", "territory_score"]
for col in norm_cols:
    mean = data[col].mean()
    std = data[col].std()
    data[col] = (data[col] - mean) / std
data

,user_id,end_date,label,launch_seq,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.844698,0.808567,0.213031,0.194954,1.038519,-0.141583,-0.955892,0.828011,-0.544818,0.746096,-1.547289,NaN
1,10000001,104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,2.431832,-2.051286,-1.254039,1.046141,-0.319111,2.055850,-1.340308,1.655552,NaN
2,10000002,132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,-1.554119,-1.372185,-2.041925,-1.046284,1.516180,1.046141,0.828011,2.055850,0.746096,1.014973,NaN
3,10000003,188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,0.194954,-0.719275,-0.813016,-0.955892,0.828011,-0.544818,0.746096,-1.211875,NaN
4,10000004,152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,0.194954,2.877345,3.917409,1.046141,-0.319111,-0.544818,-1.340308,-0.252807,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614997,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,0.194954,0.151471,-0.160653,-0.955892,-1.466234,2.055850,-1.340308,-0.287363,NaN
614998,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.8517, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",-0.721968,-2.037917,-0.321761,0.194954,-0.796952,-0.815486,-0.955892,1.975134,-0.544818,0.746096,1.864132,NaN
614999,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,NaN,NaN,NaN,0.194954,1.059136,-0.155943,1.046141,-0.319111,-0.544818,-1.340308,NaN,NaN
615000,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.0621, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",0.548020,0.670742,0.298513,2.431832,-2.052245,-1.254039,1.046141,0.828011,-0.544818,0.746096,1.541528,NaN


In [43]:
# filling null vector features with zero-vectors
data.fillna({
    "playtime_seq": str([0]*32),
    "duration_prefer": str([0]*16),
    "interact_prefer": str([0]*11)
}, inplace=True)
data

,user_id,end_date,label,launch_seq,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.844698,0.808567,0.213031,0.194954,1.038519,-0.141583,-0.955892,0.828011,-0.544818,0.746096,-1.547289,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,2.431832,-2.051286,-1.254039,1.046141,-0.319111,2.055850,-1.340308,1.655552,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,-1.554119,-1.372185,-2.041925,-1.046284,1.516180,1.046141,0.828011,2.055850,0.746096,1.014973,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000003,188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,0.194954,-0.719275,-0.813016,-0.955892,0.828011,-0.544818,0.746096,-1.211875,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,0.194954,2.877345,3.917409,1.046141,-0.319111,-0.544818,-1.340308,-0.252807,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614997,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,0.194954,0.151471,-0.160653,-0.955892,-1.466234,2.055850,-1.340308,-0.287363,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
614998,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.8517, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",-0.721968,-2.037917,-0.321761,0.194954,-0.796952,-0.815486,-0.955892,1.975134,-0.544818,0.746096,1.864132,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
614999,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN,NaN,NaN,0.194954,1.059136,-0.155943,1.046141,-0.319111,-0.544818,-1.340308,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
615000,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.0621, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",0.548020,0.670742,0.298513,2.431832,-2.052245,-1.254039,1.046141,0.828011,-0.544818,0.746096,1.541528,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [44]:
# filling null numeric features with 0
data.fillna(0, inplace=True)
data

,user_id,end_date,label,launch_seq,playtime_seq,duration_prefer,father_id_score,cast_id_score,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,205,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.844698,0.808567,0.213031,0.194954,1.038519,-0.141583,-0.955892,0.828011,-0.544818,0.746096,-1.547289,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,104,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.000000,0.000000,0.000000,2.431832,-2.051286,-1.254039,1.046141,-0.319111,2.055850,-1.340308,1.655552,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,132,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,-1.554119,-1.372185,-2.041925,-1.046284,1.516180,1.046141,0.828011,2.055850,0.746096,1.014973,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000003,188,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.000000,0.000000,0.000000,0.194954,-0.719275,-0.813016,-0.955892,0.828011,-0.544818,0.746096,-1.211875,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,152,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.000000,0.000000,0.000000,0.194954,2.877345,3.917409,1.046141,-0.319111,-0.544818,-1.340308,-0.252807,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614997,10355586,205,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.000000,0.000000,0.000000,0.194954,0.151471,-0.160653,-0.955892,-1.466234,2.055850,-1.340308,-0.287363,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
614998,10589773,210,-1,"[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.8517, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",-0.721968,-2.037917,-0.321761,0.194954,-0.796952,-0.815486,-0.955892,1.975134,-0.544818,0.746096,1.864132,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
614999,10181954,218,-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.000000,0.000000,0.000000,0.194954,1.059136,-0.155943,1.046141,-0.319111,-0.544818,-1.340308,0.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
615000,10544736,164,-1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.0621, 0, 0, 0, 0, 0, 0, 0, 0...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.0, 0.0, ...",0.548020,0.670742,0.298513,2.431832,-2.052245,-1.254039,1.046141,0.828011,-0.544818,0.746096,1.541528,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [45]:
# finally
data.loc[data.label >= 0].to_csv(output_dir + "train_data.txt", sep="\t", index=False)
data.loc[data.label < 0].to_csv(output_dir + "test_data.txt", sep="\t", index=False)